<a href="https://colab.research.google.com/github/huysuy05/LLM_Evals_On_Imbalanced_Datatset/blob/main/eval_with_hgf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
%pip install -r req.txt

Note: you may need to restart the kernel to use updated packages.


In [8]:
import torch 

print(torch.mps.is_available())

True


In [4]:
import pandas as pd

label_map = {
    0: "World",
    1: "Sports",
    2: "Business",
    3: "Sci/Tech"
}
imbalanced_data = pd.read_parquet("Data/ag_news_train_imbalanced.parquet")
balanced_data = pd.read_parquet("Data/ag_news_train_balanced.parquet")
testing_set = pd.read_parquet("Data/ag_news_test_small.parquet")
imbalanced_data_5_to_1 = pd.read_parquet("Data/ag_news_train_imbalanced_5_to_1_ratio.parquet")

balanced_data["label_word"] = balanced_data["label"].map(label_map)
imbalanced_data["label_word"] = imbalanced_data["label"].map(label_map)
testing_set["label_word"] = testing_set["label"].map(label_map)
imbalanced_data_5_to_1["label_word"] = imbalanced_data_5_to_1["label"].map(label_map)

# Shuffle the dataset
imbalanced_data = imbalanced_data.sample(frac=1).reset_index(drop=True)
balanced_data = balanced_data.sample(frac=1).reset_index(drop=True)
testing_set = testing_set.sample(frac=1).reset_index(drop=True)
imbalanced_data_5_to_1 = imbalanced_data_5_to_1.sample(frac=1).reset_index(drop=True)

imbalanced_data_5_to_1.shape


(3200, 3)

In [9]:
import pandas as pd


def classify(text, labels, model):
  """
    Use a Pipeline from Transformers for zero shot classification

    Args:
      text (str): The text data per row
      labels (List[int]): A list of label
      model (str): Model name to load
  """
  from transformers import pipeline

  classifier = pipeline("zero-shot-classification", model=model)
  result = classifier(text, labels)
  pred_label = result["labels"][0]
  return pred_label

In [12]:
from tqdm.auto import tqdm
def evaluate(data, labels, model):
  total_samples = data.shape[0]
  correct_pred = 0

  print(f"=== Starting evaluation ===")

  for idx, row in tqdm(data.iterrows()):
    text = row["text"]
    correct_label = row["label_word"].lower()
    pred_label = classify(text, labels, model)
    pred_label = pred_label.lower()

    if pred_label:
      if pred_label == correct_label:
        correct_pred += 1
        print(f"Article {idx}: Correctly classified as '{correct_label}'")
      else:
          print(f"Article {idx}: Incorrectly classified. True: '{correct_label}', Predicted: '{pred_label}'")
  acc = (correct_pred / total_samples) * 100
  return acc


In [13]:
# Test the model
labels = ["World", "Sports", "Business", "Sci/Tech"]

model = "facebook/bart-large-mnli"

print("Evaluating LLM on Balanced Data...")
balanced_accuracy = evaluate(testing_set, labels, model)
print(f"\nAccuracy on Balanced Data: {balanced_accuracy:.2f}%")

print("\n\nEvaluating LLM on Imbalanced Data with 10:1 Ratio...")
imbalanced_accuracy = evaluate(imbalanced_data, labels, model)
print(f"\nAccuracy on Imbalanced Data with 10:1 Ratio: {imbalanced_accuracy:.2f}%")

print("\n\nEvaluating LLM on Imbalanced Data with 5:1 Ratio...")
imbalanced_5_to_1_accuracy = evaluate(imbalanced_data_5_to_1, labels, model)
print(f"\nAccuracy on Imbalanced Data with 5:1 Ratio: {imbalanced_5_to_1_accuracy:.2f}%")

Evaluating LLM on Balanced Data...
=== Starting evaluation ===


0it [00:00, ?it/s]

[2025-09-16 13:09:13,075] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to mps (auto detect)


W0916 13:09:13.394000 42278 site-packages/torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
Device set to use mps:0


Article 0: Incorrectly classified. True: 'business', Predicted: 'sports'


...

Device set to use mps:0


Article 1324: Correctly classified as 'sports'


Device set to use mps:0


Article 1325: Correctly classified as 'world'


KeyboardInterrupt: 